   # Preprocessing

In [ ]:
df_rating = pd.read_csv("C:/Users/91995/Downloads/archive/ml-100k/u.data", sep="\t")
df_rating.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_rating = df_rating.drop(['timestamp'], axis=1)

rating_counts = df_rating.groupby('user_id')['rating'].count()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rating_counts.index, rating_counts.values)
plt.xlabel('user')
plt.ylabel('no of ratings')
plt.show()

In [ ]:
df_rating = df_rating.groupby('user_id').filter(lambda x: len(x) >= 270)

In [ ]:
from sklearn.preprocessing import LabelEncoder

df_user = pd.read_csv('C:/Users/91995/Downloads/archive/ml-100k/u.user', sep='|', header=None)
df_user.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']

en = LabelEncoder()
df_user['gender'] = en.fit_transform(df_user['gender'])

In [ ]:
df = pd.merge(df_rating, df_item, on='movie_id')
df = pd.merge(df, df_genre, on='movie_id')
df = pd.merge(df, df_user, on='user_id')
df = df.dropna()

In [ ]:
matrix_id = df.pivot_table(index='user_id', columns='movie_id', values='rating')

   # Model

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader, NormalPredictor
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_rating, reader)
trainData, testData = train_test_split(data, random_state=1)

param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

model = SVD(n_epochs=10, lr_all=0.005, reg_all=0.4)
model.fit(trainData)

In [ ]:
import math

def recommend(uid):
    movies_unwatched = []
    temp = matrix_id.loc[uid]
    
    for a, b in temp.iteritems():
        if math.isnan(b):
            movies_unwatched.append([a, round(model.predict(uid, a).est, 3)])
    
    movies_unwatched = sorted(movies_unwatched, key = lambda x: x[1], reverse=True)
    
    return movies_unwatched

In [ ]:
recommendations = []
for i in range(15):
    print(movies_unwatched[i][0])
    recommendations.append(links[links['movieId']==movies_unwatched[i][0]].tmdbId)

print(recommendations)